In [ ]:
import xarray as xr
import rioxarray
from pathlib import Path
import rasterio
time_series_folder = '../output_folder/completeness_analysis/'
nasa_time_series_name = '2019_SuomiNPP_nasa_time_series_fsc.nc'
meteofrance_time_series_name = '2019_meteofrance_time_series.nc'

meteofrance_time_series_path= Path(f"{time_series_folder}").joinpath(meteofrance_time_series_name)
nasa_time_series_path= Path(f"{time_series_folder}").joinpath(nasa_time_series_name)


ds = xr.open_dataset(meteofrance_time_series_path)

In [ ]:
from typing import List
from matplotlib.axes import Axes

def classes_bar_distribution(
    year_data_array: xr.DataArray, classes_to_plot: List[str] | str = "all", ax: Axes | None = None
) -> None:
    year_data_frame = year_data_array.to_pandas()

    if classes_to_plot == "all":
        classes_to_plot = year_data_frame.index

    year_data_frame = year_data_frame.transpose()
    year_data_frame.index = year_data_frame.index.strftime("%B")

    year_data_frame[classes_to_plot].plot.bar(title=f"Class distribution for year", ax=ax)

In [ ]:
print(stats.data_vars['class_distribution_area'].to_pandas().sum())

In [ ]:
classes_bar_distribution(stats.data_vars['class_distribution_percentage'], classes_to_plot=['snow_cover', 'no_snow', 'clouds', 'forest_with_snow'])

In [ ]:
classes_bar_distribution(stats.data_vars['class_distribution_area'], classes_to_plot=['snow_cover', 'no_snow', 'clouds', 'forest_with_snow'])

In [ ]:
stats.data_vars['class_distribution_percentage'].

In [ ]:
from pathlib import Path
import xarray as xr
from metrics import MeteoFranceSnowCoverProductCompleteness, NASASnowCoverProductCompleteness
time_series_folder = '../output_folder/completeness_analysis/'
nasa_time_series_name = '2021_SuomiNPP_nasa_time_series_fsc.nc'
meteofrance_time_series_name = '2021_meteofrance_time_series.nc'

meteofrance_time_series_path= Path(f"{time_series_folder}").joinpath(meteofrance_time_series_name)
nasa_time_series_path= Path(f"{time_series_folder}").joinpath(nasa_time_series_name)



ds = xr.open_dataset(meteofrance_time_series_path)#, classes=meteofrance_classes, mask_file='../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_eofr62.tiff')
metrics = MeteoFranceSnowCoverProductCompleteness(ds, mask_file='../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_eofr62.tiff')
#stats = metrics.year_statistics(months=['january', 'february'], exclude_nodata=True, netcdf_export_path='test_stats.nc', csv_export_path='test_stats.csv')

In [ ]:
ds = xr.open_dataset(nasa_time_series_path)
metrics = NASASnowCoverProductCompleteness(ds, mask_file='../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_v10_epsg4326.tiff')
stats = metrics.year_statistics(months=['january', 'february'], exclude_nodata=True, netcdf_export_path='test_stats.nc', csv_export_path='test_stats.csv')

In [ ]:
lon_min, lon_max

In [ ]:
from scipy.interpolate import griddata
import numpy as np

latitudes, longitudes = geolocation_data.data_vars['latitude'].isel(number_of_lines=slice(1500,2500), number_of_pixels=slice(5000,6000)), geolocation_data.data_vars['longitude'].isel(number_of_lines=slice(1500,2500), number_of_pixels=slice(5000,6000))
# Define the desired regular latitude-longitude grid
lat_min, lat_max = np.min(latitudes.values), np.max(latitudes.values)
lon_min, lon_max = np.min(longitudes.values), np.max(longitudes.values)
grid_lat, grid_lon = np.mgrid[
    lat_min:lat_max:0.0025,  # Adjust the resolution as needed
    lon_min:lon_max:0.0025,
]

# Interpolate sensor image onto the lat-lon grid
points = np.column_stack((latitudes.values.ravel(), longitudes.values.ravel()))
values = ndsi_snow_cover.isel(lat=slice(1500,2500), lon=slice(5000,6000)).values.ravel()
grid_image = griddata(points, values, (grid_lat, grid_lon), method="cubic")
new_image = xr.DataArray(grid_image, coords={'lat': np.arange(lat_min,lat_max,0.0025), 'lon': np.arange(lon_min,lon_max,0.0025)})
new_snow_cover = georef_data_array(new_image, 'snow_cover', crs_wkt=CRS(4326).to_wkt())
new_snow_cover.to_netcdf('test_v10_snow_cover_crop.nc')

In [ ]:
lon_max